In [ ]:
import os
import datetime
import pytz
import numpy as np
import pandas as pd
import requests
import json
from tqdm import tqdm
from MyFiles import Security


class Directory():
    def __init__(self):
        self.dt_now = datetime.datetime.now(tz= pytz.UTC)
        self.GroupDH = (str(self.dt_now.day),str(self.dt_now.month),str(self.dt_now.year),str(self.dt_now.hour),
                        str(self.dt_now.minute),str(self.dt_now.second))
        self.dh_dir ="%s_%s_%s_%s_%s_%s"%(self.GroupDH[0],self.GroupDH[1],self.GroupDH[2],self.GroupDH[3],self.GroupDH[4],self.GroupDH[5])
            
    def mkdhdir(self):
        os.mkdir(self.dh_dir) 
        os.chdir(self.dh_dir)

class EIfiles():
    def __init__(self,base_dir,dh_dir,file_name,StackSF,lenfile,countries,pollutants,ServiceUrl,EAQColm):
        self.base_dir= base_dir
        self.file_name= file_name
        self.StackSF=StackSF
        self.lenfile= lenfile
        self.countries= countries
        self.pollutants= pollutants
        self.ServiceUrl= ServiceUrl
        self.EAQColm= EAQColm
        self.dh_dir= dh_dir

    def RemoveFiles(self):  
        filesD = os.listdir()
        print('Check Files')
        for i,file in enumerate(filesD):
            if os.stat(file).st_size < self.lenfile:
                print(filesD[i],'->', os.stat(file).st_size,'Bytes')
                os.remove(filesD[i])
        return

    def down_einet(self):
        print( 'Working, whait')
        for country in tqdm(self.countries):
	        for pollutant in self.pollutants:
		        fileName = "%s_%s.csv" % (country, pollutant)
		        downloadFile = '%s/%s_%s.csv' % (self.ServiceUrl, country, pollutant)
		        file = requests.get(downloadFile).content
		        output = open(fileName, 'wb')
		        output.write(file)
		        output.close()
        return

    def AQEcsv(self):
        l_dir = os.listdir()
        columns = self.EAQColm
        DF = pd.DataFrame(columns= columns)
        for i in  range(len(l_dir)):
            Aux = pd.read_csv(l_dir[i])
            DF = pd.concat([DF, Aux])
            print(l_dir[i])
        DF.pop('network_timezone') 
        DF.to_csv('%s.csv' %( self.dh_dir))
        print('file created ', self.dh_dir,'.csv')
        os.chdir( self.base_dir)

    def down_ei(self):
        os.mkdir(self.file_name) 
        os.chdir(self.file_name)
        self.down_einet()
        self.RemoveFiles()
        self.AQEcsv()
        os.chdir(self.base_dir) 
        return

class PTFiles():
    def __init__(self,base_dir,fldTPT,StackSF,linkTPTData,linkTPTStation,TPTColm,TPTStColm,dh_dir):
        self.base_dir=base_dir
        self.fldTPT= fldTPT
        self.StackSF=StackSF
        self.linkTPTData=linkTPTData
        self.linkTPTStation=linkTPTStation
        self.TPTColm=TPTColm
        self.TPTStColm=TPTStColm
        self.dh_dir=dh_dir

    def down_TPT(self):
        os.mkdir(self.fldTPT) 
        os.chdir(self.fldTPT)
        meteo = requests.get(self.linkTPTData)
        meteo = meteo.json()            
        Samplel = list(meteo.keys())    
        TPTDf = pd.DataFrame(columns=  self.TPTColm)
        for sam in tqdm(Samplel):
            localID = list(meteo[sam].keys())
            for ID in localID:
                if type(meteo[sam][ID])==dict:
                    feature_TPT = list(meteo[sam][ID].values())
                    TPT_list=[sam, ID, feature_TPT[0], feature_TPT[1], feature_TPT[2], feature_TPT[3], feature_TPT[4], feature_TPT[5], feature_TPT[6], feature_TPT[7]]
                    TPTDf = TPTDf.append(pd.Series(TPT_list, index= self.TPTColm) , ignore_index=True)
        list_estacoes = requests.get(self.linkTPTStation)
        list_estacoes = list_estacoes.json()
        Ip_Est_Df = pd.DataFrame(columns= self.TPTStColm)
        for i in  tqdm(range(len(list_estacoes))):
            GPS = list_estacoes[i]['geometry']['coordinates']
            Id = list_estacoes[i]['properties']['idEstacao']
            Local =list_estacoes[i]['properties']['localEstacao']
            ip_est_list=[Id,Local,GPS]
            Ip_Est_Df = Ip_Est_Df.append(pd.Series(ip_est_list, index=self.TPTStColm) , ignore_index=True)
        fileName1 = "%s_%s.csv" % (self.dh_dir, self.fldTPT)
        TPTDf.to_csv(fileName1) 
        fileName2 = "%s_%s.csv" % (self.dh_dir, 'ID_estacoes_TPT')
        Ip_Est_Df.to_csv(fileName2)
        return
    
class MStack:
    def __init__(self,base_dir,file_name,StackSF,pop_cols):
        self.pop_cols=pop_cols
        self.base_dir=base_dir
        self.file_name=file_name
        self.StackSF=StackSF
        pass

    def fold(self):
        dirc = os.listdir() 
        for i in range(len(dirc)):
            if dirc[i]==self.file_name:
                return 'True'

    
    def stack(self,control=None):
        os.chdir(self.base_dir)
        dirs = os.listdir()
        ro=0
        for i in tqdm(range(len(dirs)-1)):                           
            Fol = self.fold()
            if Fol == 'True': 
                diraux = r'%s\%s\%s' % (self.base_dir, dirs[i], self.file_name)
                if self.file_name == control:
                    aux_file = r'%s_%s.csv' % (dirs[i],self.file_name)
                else:
                    aux_file = r'%s.csv' % (dirs[i])
                os.chdir(diraux)                                
                if ro==0:                                        
                    DF = pd.read_csv(aux_file)         
                    ro = DF.shape[0]
                    for col in range(len(self.pop_cols)):   
                        DF.pop(self.pop_cols[col])
                else:
                    DFaux = pd.read_csv(aux_file)     
                    ro=ro+DFaux.shape[0]
                    for col in range(len(self.pop_cols)):   
                        DFaux.pop(self.pop_cols[col])
                    DF= pd.concat([DF, DFaux])
            os.chdir(self.base_dir)
        print('total row processed:', ro)
        DF.drop_duplicates(subset=None, keep='first', inplace=True)
        print('Final Shape',DF.shape)
        os.chdir(self.base_dir)
        os.chdir(self.StackSF)
        file_name = r'Stack_%s_%s.csv' %(self.file_name,dirs[-2])
        print('Creating file:', file_name)
        DF.to_csv(file_name) 
        os.chdir(self.base_dir)

    
        
Sec=Security()
Dirfcn=Directory()
base_dir=Sec.direct
os.chdir(base_dir)
files_ei=EIfiles(base_dir,Dirfcn.dh_dir,Sec.fldAEQ,Sec.StackSF, Sec.bytes, Sec.countriesAEQ, Sec.pollutantsAEQ, Sec.linkAQE,Sec.EAQColm)
os.chdir(base_dir)
Dirfcn.mkdhdir()
files_ei.down_ei()
os.chdir(base_dir)
os.chdir(Dirfcn.dh_dir)
PTF=PTFiles(base_dir,Sec.fldTPT,Sec.StackSF,Sec.linkTPTData,Sec.linkTPTStation,Sec.TPTColm,Sec.TPTStColm,Dirfcn.dh_dir)
PTF.down_TPT()
os.chdir(base_dir)





# Call MStack Class

In [ ]:
#TPT Stack
make_stack=MStack(base_dir,Sec.fldTPT,Sec.StackSF,Sec.pop_cols_TPT)
os.chdir(base_dir)
make_stack.stack(Sec.fldTPT)
os.chdir(base_dir)

#EIO Stack
make_stack=MStack(base_dir,Sec.fldAEQ,Sec.StackSF,Sec.pop_cols_EIO)
os.chdir(base_dir)
make_stack.stack()
os.chdir(base_dir)